Training Notebook for the G2Net competition. This implements a Bi-directional GRU using Keras, using preprocessed spectrogram.

This uses Yasufumi Nakama's spectrogram preprocessing notebooks and datasets:
* Train: [Notebook](https://www.kaggle.com/yasufuminakama/g2net-spectrogram-generation-train), [Dataset](https://www.kaggle.com/yasufuminakama/g2net-n-mels-128-train-images)
* Test: [Notebook](https://www.kaggle.com/yasufuminakama/g2net-spectrogram-generation-test), [Dataset](https://www.kaggle.com/yasufuminakama/g2net-n-mels-128-test-images)

In [1]:
import os

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
class CustomDataset(tf.keras.utils.Sequence):
    def __init__(self, df, directory, batch_size=32, random_state=42, shuffle=True, target=True, ext='.npy'):
        np.random.seed(random_state)
        
        self.directory = directory
        self.df = df
        self.shuffle = shuffle
        self.target = target
        self.batch_size = batch_size
        self.ext = ext
        
        self.on_epoch_end()
    
    def __len__(self):
        return np.ceil(self.df.shape[0] / self.batch_size).astype(int)
    
    def __getitem__(self, idx):
        start_idx = idx * self.batch_size
        batch = self.df[start_idx: start_idx + self.batch_size]
        
        signals = []

        for fname in batch.id:
            path = os.path.join(self.directory, fname + self.ext)
            data = np.load(path)
            signals.append(data)
        
        signals = np.stack(signals).astype('float32')
        
        if self.target:
            return signals, batch.target.values
        else:
            return signals
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

In [3]:
def build_model():
    inputs = layers.Input(shape=(27, 128))

    gru1 = layers.Bidirectional(layers.GRU(128, return_sequences=True), name='gru_1')
    gru2 = layers.Bidirectional(layers.GRU(128, return_sequences=True), name='gru_2')
    pool1 = layers.GlobalAveragePooling1D(name='avg_pool')
    pool2 = layers.GlobalMaxPooling1D(name='max_pool')

    x = gru1(inputs)
    x = gru2(x)
    x = tf.keras.layers.Concatenate()([pool1(x), pool2(x)])
    
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dense(128, activation="relu")(x)
    x = layers.Dense(1, activation="sigmoid", name="sigmoid")(x)

    model = tf.keras.Model(inputs=inputs, outputs=x)
    
    return model

In [4]:
train = pd.read_csv('./input/g2net-gravitational-wave-detection/training_labels.csv')
sub = pd.read_csv('./input/g2net-gravitational-wave-detection/sample_submission.csv')
train.head()

,id,target
0,00000e74ad,1
1,00001f4945,0
2,0000661522,0
3,00007a006a,0
4,0000a38978,1


In [5]:
sample_df = train.sample(frac=1).reset_index(drop=True)

split = int(sample_df.shape[0] * 0.8)
train_df = sample_df[:split]
valid_df = sample_df[split:]

In [6]:
train_dset = CustomDataset(
    train_df, './input/g2net-n-mels-128-train-images', batch_size=64)

valid_dset = CustomDataset(
    valid_df, './input/g2net-n-mels-128-train-images', batch_size=64, shuffle=False)

test_dset = CustomDataset(
    sub, "./input/g2net-n-mels-128-test-images", batch_size=64, target=False, shuffle=False)

In [7]:
model = build_model()
model.compile("adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC()])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 27, 128)]    0                                            
__________________________________________________________________________________________________
gru_1 (Bidirectional)           (None, 27, 256)      198144      input_1[0][0]                    
__________________________________________________________________________________________________
gru_2 (Bidirectional)           (None, 27, 256)      296448      gru_1[0][0]                      
__________________________________________________________________________________________________
avg_pool (GlobalAveragePooling1 (None, 256)          0           gru_2[0][0]                      
______________________________________________________________________________________________

In [8]:
ckpt = tf.keras.callbacks.ModelCheckpoint(
    "model_weights.h5", save_best_only=True, save_weights_only=True,
)

train_history = model.fit(
    train_dset, 
    use_multiprocessing=True, 
    workers=4, 
    epochs=10,
    validation_data=valid_dset,
    callbacks=[ckpt],
)

Epoch 1/10
7000/7000 [==============================] - 1347s 191ms/step - loss: 0.6662 - auc: 0.5997 - val_loss: 0.5844 - val_auc: 0.7542
Epoch 2/10
7000/7000 [==============================] - 1345s 192ms/step - loss: 0.5613 - auc: 0.7599 - val_loss: 0.5352 - val_auc: 0.7947
Epoch 3/10
7000/7000 [==============================] - 1398s 200ms/step - loss: 0.5436 - auc: 0.7786 - val_loss: 0.5588 - val_auc: 0.7998
Epoch 4/10
7000/7000 [==============================] - 1387s 198ms/step - loss: 0.5393 - auc: 0.7822 - val_loss: 0.5280 - val_auc: 0.7984
Epoch 5/10
7000/7000 [==============================] - 1269s 181ms/step - loss: 0.5371 - auc: 0.7846 - val_loss: 0.5252 - val_auc: 0.7963
Epoch 6/10
7000/7000 [==============================] - 1286s 184ms/step - loss: 0.5410 - auc: 0.7814 - val_loss: 0.5344 - val_auc: 0.7927
Epoch 7/10
7000/7000 [==============================] - 1287s 184ms/step - loss: 0.5441 - auc: 0.7785 - val_loss: 0.5542 - val_auc: 0.7712
Epoch 8/10
7000/7000 [=====

In [9]:
model.load_weights('model_weights.h5')

In [10]:
y_pred = model.predict(
    test_dset, use_multiprocessing=True, workers=4, verbose=1
)

3532/3532 [==============================] - 469s 132ms/step


In [11]:
sub['target'] = y_pred
sub.to_csv('submission.csv', index=False)